In [5]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
from IPython.display import Video

# 1. 모델 로드
model = joblib.load("output/rf_model.pkl")
scaler = joblib.load("output/scaler.pkl")

# 2. MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 3. 동영상 파일 열기
cap = cv2.VideoCapture("UCF_Crime_Split/test/fighting/Fighting011_x264.mp4")

# 4. 저장할 비디오 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output/result.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if results.pose_landmarks:
        keypoints = []
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])

        pose_coords = np.array(keypoints).reshape(1, -1)
        pose_coords = scaler.transform(pose_coords)
        pred = model.predict(pose_coords)
        
        if pred != "normal":
            h, w = frame.shape[:2]
            font_scale = min(w, h) / 600
            thickness = max(1, int(font_scale * 2))
            cv2.putText(
                frame,
                f'Prediction: {pred}',
                (int(w*0.05), int(h*0.1)),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale,
                (0, 0, 255),
                thickness
            )
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    out.write(frame)

cap.release()
out.release()

I0000 00:00:1748595017.337174  588901 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1748595017.373059  966363 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce RTX 2080 Ti/PCIe/SSE2
W0000 00:00:1748595017.472627  966315 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748595017.529801  966339 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
